In [1]:
# based on https://www.toptal.com/apache/apache-spark-streaming-twitter
import socket
import sys
import requests
import requests_oauthlib
import json
from mytwitter import CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET

In [2]:
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

In [3]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-88.11,35.32,-86.45,36.35'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

In [4]:
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']+ '\n'
            print("Tweet Text: " + tweet_text)
            #print(type(tweet_text))
            #print(type(tcp_connection))            
            tcp_connection.send(tweet_text.encode())
        except:          
            print("Error")

In [5]:
resp = get_tweets()

https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-88.11,35.32,-86.45,36.35&track=# <Response [200]>


In [6]:
TCP_IP = "localhost"
TCP_PORT = 5045
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()

Waiting for TCP connection...
Connected... Starting getting tweets.
https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-88.11,35.32,-86.45,36.35&track=# <Response [200]>


In [7]:
send_tweets_to_spark(resp, conn)

Tweet Text: @TweetTylerOrr Omg lol quality time is how I feel loved and acts of service is how I show love. You’re so good at this game 😂

Tweet Text: @skye_daddy No, I’m both proud and grateful for being able to find a way up and for there being a path to begin wit… https://t.co/lQffVWxnN3

Tweet Text: no THEY understood the assignment https://t.co/CLf3L1rlBp

Tweet Text: @Heidiaca That’s me in heels 😂😂

Tweet Text: Finally! A grocery store Downtown! More from @jenniferbiggs1: https://t.co/kXaIrTY9t3

Tweet Text: Why am I only now finding @MapleStreetBisc #comfortfood https://t.co/FPHX2u7hTJ

Tweet Text: one step at a time 💕

Tweet Text: @DrewFos10875444 Congrats!

Error
Tweet Text: @TheDrunkenFins @MiamiDolphins Awesome pic

Tweet Text: When you’re supposed to shop for a bday gift for your sister and you get distracted and order something for yoursel… https://t.co/6gRflfwaCT

Tweet Text: Here’s a song for you… Fade To Black (Live in Mexico City) by Metallica
https://t.co/rU00zvoWJJ



KeyboardInterrupt: 